In [1]:
import numpy as np
import pandas as pd
import pdg
from fractions import Fraction
import itertools

api = pdg.connect()

## [PDG dataset](https://pdg.lbl.gov/)

I use PDG's python api to get the main data. Here is some code showing how to use the api.

In [2]:
p = api.get_particle_by_name('p')
print(p.name,p.mass,p.charge,p.quantum_I,p.quantum_J)

Sigma_puls = api.get_particle_by_name('Sigma+')
print(Sigma_puls.name,Sigma_puls.mass,Sigma_puls.charge,Sigma_puls.quantum_I,Sigma_puls.quantum_J)

p 0.93827208816 1.0 1/2 1/2
Sigma+ 1.18937004127302 None 1 1/2


## My own addition

However PDG do not give some simple informaition such as Isospin third component $I_3$.

In [3]:
path = 'Dataset.xlsx'
selfdefine = pd.read_excel(path)
selfdefine.index=selfdefine['name'].values
selfdefine

,name,mass,J,B,S,Q,I,I3
pi+,pi+,139.570,0,0.0,0,1.0,1,1
pi0,pi0,134.980,0,0.0,0,0.0,1,0
pi-,pi-,139.570,0,0.0,0,-1.0,1,-1
K+,K+,493.690,0,0.0,1,1.0,1/2,+1/2
K0,K0,497.690,0,0.0,1,0.0,1/2,-1/2
K-,K-,493.680,0,0.0,-1,-1.0,1/2,-1/2
-K0,-K0,497.690,0,0.0,-1,0.0,1/2,+1/2
eta,eta,547.300,0,0.0,0,0.0,0,0
p,p,938.272,1/2,1.0,0,1.0,1/2,+1/2
n,n,939.567,1/2,1.0,0,0.0,1/2,-1/2


You can get data for a paticular particle by this way

In [4]:
selfdefine.loc['p']

name          p
mass    938.272
J           1/2
B           1.0
S             0
Q           1.0
I           1/2
I3         +1/2
Name: p, dtype: object

You can search particles with paticular property by this way

In [5]:
selfdefine.loc[selfdefine['Q'] == 1]

,name,mass,J,B,S,Q,I,I3
pi+,pi+,139.570,0,0.0,0,1.0,1,1
K+,K+,493.690,0,0.0,1,1.0,1/2,+1/2
p,p,938.272,1/2,1.0,0,1.0,1/2,+1/2
Sigma+,Sigma+,1189.370,1/2,1.0,-1,1.0,1,1


## My reaction checker

I will primarily utilize PDG's data. However, for $S$ and $I_3$, I rely on my own dataset.

This task proves more challenging than anticipated, as PDG's dataset API does not provide all the required data. Additionally, verifying $I_3$ violation necessitates expertise in angular momentum coupling techniques.

The names of particles align primarily with PDG conventions. However, for anti-particles, I prepend a '-' before their names. Regarding neutrinos, PDG combines three flavors into one name, 'mu', perhaps due to neutrino oscillation. For the verification of lepton number, I opt to use 'nu_e', 'nu_nu', and 'nu_tau'.

In [6]:
def get_baryon_number(name):
    # a small assistant function to get baryon number
    if api.get_particle_by_name(name).is_baryon:
        return 1
    if api.get_particle_by_name(name).is_quark:
        return 1/3
    return 0

def get_lepton_number(name):
    if name in ['e','nu_e','e-']:
        L_e=1
    elif name in ['e+']:
        L_e=-1
    else:
        L_e=0
    if name in ['mu','nu_mu','mu-']:
        L_mu=1
    elif name in ['mu+']:
        L_mu=-1
    else:
        L_mu=0
    if name in ['tau','nu_tau','tau-']:
        L_tau=1
    elif name in ['tau+']:
        L_tau=-1
    else:
        L_tau=0
    return L_e,L_mu,L_tau

def convert_parity(parity): #'+' to 1, '-' to -1 
    if parity=='+':
        return 1
    if parity=='-':
        return -1
    return None
convert_parity = np.vectorize(convert_parity)

# convert_bool=lambda x:False if x=='False' else True
# convert_bool = np.vectorize(convert_bool)

def get_isospin(name):
    if name == 'gamma':
        return 0
    # In PDG's data, the isospin of gamma is '0,1'
    

    isospin = api.get_particle_by_name(name).quantum_I
    if isospin == None:
        return 0
    return Fraction(isospin)

def sum_isospin(I):
    # I is a list of isospin I not I3
    # get the result of all possible total isospin
    # In order to avoid floating-point errors, the result is doubled to discribe the half-interger well.
    I = np.array(I)
    possible_result = set()
    for i in itertools.product([-2, 2], repeat=len(I)):
        possible_result.add(int(abs((I*np.array(i)).sum())))
    return possible_result

def check_isospin(I_reactant,I_product):
    return bool(sum_isospin(I_reactant) & sum_isospin(I_product))

def get_charge(name):
    # As you see, PDG think that the charge of 'Sigma+' is None. So strange!
    PDGdata = api.get_particle_by_name(name)
    is_hardon = PDGdata.is_baryon or PDGdata.is_meson
    if is_hardon:
        return selfdefine.loc[name]['Q']
    if PDGdata.charge==None:
        return 0
    return PDGdata.charge    


def find_subscript_index(input_string):
    if input_string[:2]=='nu':
        return input_string.find('_')
    for i in range(len(input_string) - 1, -1, -1):
        if input_string[i].isalpha():
            return i+1
    return 0

def get_particle_Latex(name):
    # from particle name in ASCII code to LaTeX code. For example, 'pi+' to '\pi^+'
    anti = ''
    if name[0]=='-':
        anti = r'\bar '
        name=name[1:]
    index = 0
    if name[:2]=='nu':
        index = name.find('_')
        nametext = name[:index]
        subscript = name[index+1:]
        nametext = '\\' + nametext
        if len(subscript)>1:
            subscript = '\\' + subscript
        final = anti + nametext+'_'+subscript
    else:
        for i in range(len(name) - 1, -1, -1):
            if name[i].isalpha():
                index = i
                break
        nametext = name[:index+1]
        subscript = name[index+1:]
        if len(nametext)>1:
            nametext = '\\' + nametext
        if subscript:
            subscript = '^'+subscript
        final = anti + nametext +subscript
    return final


def get_reaction_Latex(reactant,product):
    # from reaction imformation to get the equation for reaction in latex
    reactantlatex_list=[]
    for i in reactant:
        reactantlatex_list.append(get_particle_Latex(i))
    productlatex_list=[]
    for i in product:
        productlatex_list.append(get_particle_Latex(i))

    reactantlatex = ' + '.join(reactantlatex_list)
    productlatex = ' + '.join(productlatex_list)
    return reactantlatex+r' \to '+productlatex

def reaction_checker(reactant,product):
    # print('CCCCCCCCCCCCHHHHHHHHHHH')
    # get in reaction datas
    reactant_df = pd.DataFrame({'name':reactant})
    reactant_df.index = [f'reactant{i+1}' for i in range(len(reactant))]
    product_df = pd.DataFrame({'name':product})
    product_df.index = [f'product{i+1}' for i in range(len(product))]

    # anti-particle, because PDG do not give some data about anti-particles
    anti_reactant = [1 for _ in range(len(reactant))]
    for i in range(len(reactant)):
        if reactant[i][0]=='-':
            anti_reactant[i]=-1
            reactant[i]=reactant[i][1:]
    anti_reactant = np.array(anti_reactant)
    anti_product = [1 for _ in range(len(product))]
    for i in range(len(product)):
        if product[i][0]=='-':
            anti_product[i]=-1
            product[i]=product[i][1:]
    anti_product = np.array(anti_product)

    # neutrino: PDG gives three flavors of neutrino in one particle data 'nu'
    # don't worry, the information with neutrino flavor is backed up and seemingly only used in lepton number
    neu_reactant = reactant[:]
    for i in range(len(reactant)):
        if reactant[i][:2]=='nu':
            reactant[i]='nu'
    neu_product = product[:]
    for i in range(len(product)):
        if product[i][:2]=='nu':
            product[i]='nu'
    # print(reactant,product)
    # print(neu_reactant,neu_product)
    # print('CCCCCCCCCCCCHHHHHHHHHHH')

    # check em-charge
    reactant_df['Q']=[get_charge(i) for i in reactant]
    product_df['Q']=[get_charge(i) for i in product]
    reactant_df['Q']*=anti_reactant
    product_df['Q']*=anti_product
    # print('CCCCCCCCCCCCHHHHHHHHHHH')
    check_charge = ((abs(reactant_df['Q'].values.sum()-product_df['Q'].values.sum()))<1e-3)
    # print('CCCCCCCCCCCCHHHHHHHHHHH')
    # check baryon number
    reactant_df['B']=[get_baryon_number(i) for i in reactant]
    product_df['B']=[get_baryon_number(i) for i in product]
    reactant_df['B']*=anti_reactant
    product_df['B']*=anti_product
    check_B = ((abs(reactant_df['B'].values.sum()-product_df['B'].values.sum()))<1e-3)

    # check lepton number
    reactant_df['L_e'],reactant_df['L_mu'],reactant_df['L_tau']=[[get_lepton_number(row)[i] for row in neu_reactant] for i in range(3)]
    product_df['L_e'],product_df['L_mu'],product_df['L_tau']=[[get_lepton_number(row)[i] for row in neu_product] for i in range(3)]
    
    reactant_df['L_e']*=anti_reactant
    product_df['L_e']*=anti_product
    reactant_df['L_mu']*=anti_reactant
    product_df['L_mu']*=anti_product
    reactant_df['L_tau']*=anti_reactant
    product_df['L_tau']*=anti_product

    check_L_e = (reactant_df['L_e'].values.sum()==product_df['L_e'].values.sum())
    check_L_mu = (reactant_df['L_mu'].values.sum()==product_df['L_mu'].values.sum())
    check_L_tau = (reactant_df['L_tau'].values.sum()==product_df['L_tau'].values.sum())
    # print(check_L_e,check_L_mu,check_L_tau)

    # check strange number
    reactant_df['S']=[(0 if not(i in set(selfdefine.index.values)) 
                       else selfdefine.loc[i]['S'])
                       for i in reactant]
    product_df['S']=[(0 if not(i in set(selfdefine.index.values)) 
                       else selfdefine.loc[i]['S'])
                       for i in product]
    reactant_df['S']*=anti_reactant
    product_df['S']*=anti_product
    check_S = ((abs(reactant_df['S'].values.sum()-product_df['S'].values.sum()))<1e-3)


    # check isospin: total isospin cannot be added directly
    reactant_df['I']=[get_isospin(i) for i in reactant]
    product_df['I']=[get_isospin(i) for i in product]
    check_I = check_isospin(reactant_df['I'].values,product_df['I'].values)

    # check I3
    reactant_df['I3']=[(0 if not(i in set(selfdefine.index.values)) 
                       else Fraction(selfdefine.loc[i]['I3']))
                       for i in reactant]
    product_df['I3']=[(0 if not(i in set(selfdefine.index.values)) 
                       else Fraction(selfdefine.loc[i]['I3']))
                       for i in product]
    reactant_df['I3']*=anti_reactant
    product_df['I3']*=anti_product
    reactant_df.fillna(0, inplace=True)
    product_df.fillna(0, inplace=True)
    check_I3 = ((abs(reactant_df['I3'].values.sum()-product_df['I3'].values.sum()))<1e-3)

    # check C-parity
    reactant_df['C']=[api.get_particle_by_name(i).quantum_C for i in reactant]
    product_df['C']=[api.get_particle_by_name(i).quantum_C for i in product]
    if (None in reactant_df['C'].values) or (None in product_df['C'].values):
        check_C = None
    else:
        # pass
        reactant_parity = convert_parity(reactant_df['C']).prod()
        product_parity = convert_parity(product_df['C']).prod()
        check_C=(reactant_parity == product_parity)

    # check G-parity
    reactant_df['G']=[api.get_particle_by_name(i).quantum_G for i in reactant]
    product_df['G']=[api.get_particle_by_name(i).quantum_G for i in product]
    if (None in reactant_df['G'].values) or (None in product_df['G'].values):
        check_G = None
    else:
        # pass
        reactant_parity = convert_parity(reactant_df['G']).prod()
        product_parity = convert_parity(product_df['G']).prod()
        check_G=(reactant_parity == product_parity)

    
    # form result
    analy_df=pd.concat([reactant_df,product_df])
    analy_df.loc['checking']=['checking',
                              str(check_charge),
                              str(check_B),
                              str(check_L_e),
                              str(check_L_mu),
                              str(check_L_tau),
                              str(check_S),
                              str(check_I),
                              str(check_I3),
                              str(check_C),
                              str(check_G)]
    
    
    Possibility = 'Yes'
    strong = 'Yes'
    EM = 'Yes'
    weak = 'Yes'
    # checking weather and how the reaction can happen
    possi = analy_df.loc['checking']['Q':'L_tau']
    violations = possi[possi=='False']
    if len(violations)!=0:
        Possibility=','.join(violations.index)+' violation'
        strong = 'No'
        EM = 'No'
        weak = 'No'

    if Possibility=='Yes':
        # checking strong interaction
        strong_data = analy_df.loc['checking']['S':'G']
        violations = strong_data[strong_data=='False']
        if len(violations)!=0:
            strong =','.join(violations.index)+' violation'

        # checking EM interaction
        EM_data = analy_df.loc['checking'][['S','I3','C']]
        violations = EM_data[EM_data=='False']
        if len(violations)!=0:
            EM =','.join(violations.index)+' violation'
        

    Checking_result = pd.DataFrame({'reaction':['$'+get_reaction_Latex(reactant_df['name'].values,product_df['name'].values)+'$'],
                                    'Possibility':[Possibility],
                                    'Strong':[strong],
                                    'EM':[EM],
                                    'Weak':[weak]})


    return analy_df,Checking_result



## HW

My reaction checker generates two lists: one containing reactants and the other containing products.

It returns two dataframes. The first dataframe indicates the violated quantum numbers and the nature of the violations. The second dataframe provides information on the types of interactions in which the reaction can occur.

#### 1
$$
p+p\to n+\pi^++p	
$$

In [7]:
reactant = ['p','p']
product = ['n','pi+','p']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)

total_result = pd.DataFrame()
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,p,1.0,1,0,0,0,0,1/2,1/2,None,None
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,n,0.0,1,0,0,0,0,1/2,-1/2,None,None
product2,pi+,1.0,0,0,0,0,0,1,1,None,-
product3,p,1.0,1,0,0,0,0,1/2,1/2,None,None
checking,checking,True,True,True,True,True,True,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$p + p \to n + \pi^+ + p$,Yes,Yes,Yes,Yes


#### 2
$$
p+p \rightarrow \overline{{n}}+p+p+\pi^0
$$

In [8]:
reactant = ['p','p']
product = ['-n','pi+','p','pi0']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,p,1.0,1,0,0,0,0,1/2,1/2,None,None
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,-n,-0.0,-1,0,0,0,0,1/2,1/2,None,None
product2,pi+,1.0,0,0,0,0,0,1,1,None,-
product3,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product4,pi0,0.0,0,0,0,0,0,1,0,+,-
checking,checking,True,False,True,True,True,True,True,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$p + p \to \bar n + \pi^+ + p + \pi^0$,B violation,No,No,No


#### 3
$$
\pi^++p\to \Sigma^++p
$$

In [9]:
reactant = ['pi+','p']
product = ['Sigma+','p']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi+,1.0,0,0,0,0,0,1,1,None,-
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Sigma+,1.0,1,0,0,0,-1,1,1,None,None
product2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
checking,checking,True,False,True,True,True,False,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^+ + p \to \Sigma^+ + p$,B violation,No,No,No


#### 4
$$
\pi^++p\to \Sigma^++K^+
$$

In [10]:
reactant = ['pi+','p']
product = ['Sigma+','K+']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi+,1.0,0,0,0,0,0,1,1,None,-
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Sigma+,1.0,1,0,0,0,-1,1,1,None,None
product2,K+,1.0,0,0,0,0,1,1/2,1/2,None,None
checking,checking,True,True,True,True,True,True,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^+ + p \to \Sigma^+ + K^+$,Yes,Yes,Yes,Yes


#### 5
$$
\pi^++p\to \Sigma^++p+\pi^0
$$

In [11]:
reactant = ['pi+','p']
product = ['Sigma+','p','pi0']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi+,1.0,0,0,0,0,0,1,1,None,-
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Sigma+,1.0,1,0,0,0,-1,1,1,None,None
product2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product3,pi0,0.0,0,0,0,0,0,1,0,+,-
checking,checking,True,False,True,True,True,False,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^+ + p \to \Sigma^+ + p + \pi^0$,B violation,No,No,No


#### 6
$$
\pi^++p\to \Xi^0+\cdots
$$

In [12]:
reactant = ['pi+','p']
product = ['Xi0']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi+,1.0,0,0,0,0,0,1,1,None,-
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Xi0,0.0,1,0,0,0,-2,1/2,1/2,None,None
checking,checking,False,True,True,True,True,False,True,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^+ + p \to \Xi^0$,Q violation,No,No,No


#### 7
$$
\Sigma^++p\to \Lambda^0+\pi^++p
$$

In [13]:
reactant = ['Sigma+','p']
product = ['Lambda0','pi+','p']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Sigma+,1.0,1,0,0,0,-1,1,1,None,None
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,pi+,1.0,0,0,0,0,0,1,1,None,-
product3,p,1.0,1,0,0,0,0,1/2,1/2,None,None
checking,checking,True,True,True,True,True,True,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Sigma^+ + p \to \Lambda^0 + \pi^+ + p$,Yes,Yes,Yes,Yes


#### 8
$$
\Sigma^-+p\to \Xi^-+p
$$

In [14]:
reactant = ['Sigma-','p']
product = ['Xi-','p']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Sigma-,-1.0,1,0,0,0,-1,1,-1,None,None
reactant2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product1,Xi-,-1.0,1,0,0,0,-2,1/2,-1/2,None,None
product2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Sigma^- + p \to \Xi^- + p$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 9
$$
\Xi^0 \to e^- + p + \bar\nu_e
$$

In [15]:
reactant = ['Xi0']
product = ['e-','p','-nu_e']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Xi0,0.0,1,0,0,0,-2,1/2,1/2,None,None
product1,e-,-1.0,0,1,0,0,0,0,0,None,None
product2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product3,-nu_e,-0.0,0,-1,0,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Xi^0 \to e^- + p + \bar \nu_e$,Yes,S violation,S violation,Yes


#### 10
$$
\Xi^0 \to \Lambda^0 + \pi^0
$$

In [16]:
reactant = ['Xi0']
product = ['Lambda0','pi0']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Xi0,0.0,1,0,0,0,-2,1/2,1/2,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,pi0,0.0,0,0,0,0,0,1,0,+,-
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Xi^0 \to \Lambda^0 + \pi^0$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 11
$$
\Xi^- \to \Lambda^0 + \pi^-
$$

In [17]:
reactant = ['Xi-']
product = ['Lambda0','pi-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Xi-,-1.0,1,0,0,0,-2,1/2,-1/2,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,pi-,-1.0,0,0,0,0,0,1,-1,None,-
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Xi^- \to \Lambda^0 + \pi^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


## 12
$$
\Omega^- \to \Lambda^0 + \pi^-
$$

In [18]:
reactant = ['Omega-']
product = ['Lambda0','pi-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Omega-,-1.0,1,0,0,0,-3,0,0,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,pi-,-1.0,0,0,0,0,0,1,-1,None,-
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Omega^- \to \Lambda^0 + \pi^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 13
$$
\Omega^- \to \Lambda^0 + K^-
$$

In [19]:
reactant = ['Omega-']
product = ['Lambda0','K-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Omega-,-1.0,1,0,0,0,-3,0,0,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,K-,-1.0,0,0,0,0,-1,1/2,-1/2,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Omega^- \to \Lambda^0 + K^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 14
$$
\Omega^- \to \Xi^0 + \pi^-
$$

In [20]:
reactant = ['Omega-']
product = ['Xi0','pi-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Omega-,-1.0,1,0,0,0,-3,0,0,None,None
product1,Xi0,0.0,1,0,0,0,-2,1/2,1/2,None,None
product2,pi-,-1.0,0,0,0,0,0,1,-1,None,-
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Omega^- \to \Xi^0 + \pi^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 15
$$
\Omega^- \to \Xi^0 + K^-
$$

In [21]:
reactant = ['Omega-']
product = ['Xi0','K-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Omega-,-1.0,1,0,0,0,-3,0,0,None,None
product1,Xi0,0.0,1,0,0,0,-2,1/2,1/2,None,None
product2,K-,-1.0,0,0,0,0,-1,1/2,-1/2,None,None
checking,checking,True,True,True,True,True,True,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Omega^- \to \Xi^0 + K^-$,Yes,Yes,Yes,Yes


#### 16
$$
\Xi^- \to \Sigma^0 + \mu^- + \bar\nu_\mu
$$

In [22]:
reactant = ['Xi-']
product = ['Sigma0','mu-','-nu_mu']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Xi-,-1.0,1,0,0,0,-2,1/2,-1/2,None,None
product1,Sigma0,0.0,1,0,0,0,-1,1,0,None,None
product2,mu-,-1.0,0,0,1,0,0,0,0,None,None
product3,-nu_mu,-0.0,0,0,-1,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Xi^- \to \Sigma^0 + \mu^- + \bar \nu_\mu$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 17
$$
\Xi^- \to e^- + p + \bar\nu_e + \pi^-
$$

In [23]:
reactant = ['Xi-']
product = ['e-','p','-nu_e','pi-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Xi-,-1.0,1,0,0,0,-2,1/2,-1/2,None,None
product1,e-,-1.0,0,1,0,0,0,0,0,None,None
product2,p,1.0,1,0,0,0,0,1/2,1/2,None,None
product3,-nu_e,-0.0,0,-1,0,0,0,0,0,None,None
product4,pi-,-1.0,0,0,0,0,0,1,-1,None,-
checking,checking,True,True,True,True,True,False,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Xi^- \to e^- + p + \bar \nu_e + \pi^-$,Yes,S violation,S violation,Yes


#### 18
$$
\Sigma^0 \to \Lambda^0 + \gamma + \gamma
$$

In [24]:
reactant = ['Sigma0']
product = ['Lambda0','gamma','gamma']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Sigma0,0.0,1,0,0,0,-1,1,0,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,gamma,0.0,0,0,0,0,0,0,0,-,None
product3,gamma,0.0,0,0,0,0,0,0,0,-,None
checking,checking,True,True,True,True,True,True,False,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Sigma^0 \to \Lambda^0 + \gamma + \gamma$,Yes,I violation,Yes,Yes


#### 19
$$
\Sigma^0 \to \Lambda^0 + e^- + e^+
$$

In [25]:
reactant = ['Sigma0']
product = ['Lambda0','e-','e+']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,Sigma0,0.0,1,0,0,0,-1,1,0,None,None
product1,Lambda0,0.0,1,0,0,0,-1,0,0,None,None
product2,e-,-1.0,0,1,0,0,0,0,0,None,None
product3,e+,1.0,0,-1,0,0,0,0,0,None,None
checking,checking,True,True,True,True,True,True,False,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\Sigma^0 \to \Lambda^0 + e^- + e^+$,Yes,I violation,Yes,Yes


#### 20
$$
K^+ \to \mu^+ + \nu_\mu
$$

In [26]:
reactant = ['K+']
product = ['mu+','nu_mu']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,K+,1.0,0,0,0,0,1,1/2,1/2,None,None
product1,mu+,1.0,0,0,-1,0,0,0,0,None,None
product2,nu_mu,0.0,0,0,1,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$K^+ \to \mu^+ + \nu_\mu$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 21
$$
K^- \to \mu^- + \bar \nu_\mu
$$

In [27]:
reactant = ['K-']
product = ['mu-','-nu_mu']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,K-,-1.0,0,0,0,0,-1,1/2,-1/2,None,None
product1,mu-,-1.0,0,0,1,0,0,0,0,None,None
product2,-nu_mu,-0.0,0,0,-1,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$K^- \to \mu^- + \bar \nu_\mu$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 22
$$
K^0 \to \pi^+ + \pi^-
$$

In [28]:
reactant = ['K0']
product = ['pi+','pi-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,K0,0.0,0,0,0,0,1,1/2,-1/2,None,None
product1,pi+,1.0,0,0,0,0,0,1,1,None,-
product2,pi-,-1.0,0,0,0,0,0,1,-1,None,-
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$K^0 \to \pi^+ + \pi^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 23
$$
K^0 \to \mu^+ + \mu^-
$$

In [29]:
reactant = ['K0']
product = ['mu+','mu-']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,K0,0.0,0,0,0,0,1,1/2,-1/2,None,None
product1,mu+,1.0,0,0,-1,0,0,0,0,None,None
product2,mu-,-1.0,0,0,1,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$K^0 \to \mu^+ + \mu^-$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 24

$$
K^+ \to \pi^0 + e^+ + \nu_e
$$

In [30]:
reactant = ['K+']
product = ['pi0','e+','nu_e']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,K+,1.0,0,0,0,0,1,1/2,1/2,None,None
product1,pi0,0.0,0,0,0,0,0,1,0,+,-
product2,e+,1.0,0,-1,0,0,0,0,0,None,None
product3,nu_e,0.0,0,1,0,0,0,0,0,None,None
checking,checking,True,True,True,True,True,False,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$K^+ \to \pi^0 + e^+ + \nu_e$,Yes,"S,I,I3 violation","S,I3 violation",Yes


#### 25
$$
\mu^+ \to e^+ + \nu_e
$$

In [31]:
reactant = ['mu+']
product = ['e+','nu_e']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,mu+,1.0,0,0,-1,0,0,0,0,None,None
product1,e+,1.0,0,-1,0,0,0,0,0,None,None
product2,nu_e,0.0,0,1,0,0,0,0,0,None,None
checking,checking,True,True,True,False,True,True,True,True,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\mu^+ \to e^+ + \nu_e$,L_mu violation,No,No,No


#### 26
$$
\pi^+ \to \mu^+ + \bar \nu_\mu
$$

In [32]:
reactant = ['pi+']
product = ['mu+','-nu_mu']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi+,1.0,0,0,0,0,0,1,1,None,-
product1,mu+,1.0,0,0,-1,0,0,0,0,None,None
product2,-nu_mu,-0.0,0,0,-1,0,0,0,0,None,None
checking,checking,True,True,True,False,True,True,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^+ \to \mu^+ + \bar \nu_\mu$,L_mu violation,No,No,No


#### 27
$$
\pi^- \to \mu^- + \bar \nu_\mu
$$

In [33]:
reactant = ['pi-']
product = ['mu-','-nu_mu']
reaction,result=reaction_checker(reactant,product)
display(reaction,result)
total_result = pd.concat([total_result, result], ignore_index=True)

,name,Q,B,L_e,L_mu,L_tau,S,I,I3,C,G
reactant1,pi-,-1.0,0,0,0,0,0,1,-1,None,-
product1,mu-,-1.0,0,0,1,0,0,0,0,None,None
product2,-nu_mu,-0.0,0,0,-1,0,0,0,0,None,None
checking,checking,True,True,True,True,True,True,False,False,None,None


,reaction,Possibility,Strong,EM,Weak
0,$\pi^- \to \mu^- + \bar \nu_\mu$,Yes,"I,I3 violation",I3 violation,Yes


In [34]:
total_result.index+=1
total_result

,reaction,Possibility,Strong,EM,Weak
1,$p + p \to n + \pi^+ + p$,Yes,Yes,Yes,Yes
2,$p + p \to \bar n + \pi^+ + p + \pi^0$,B violation,No,No,No
3,$\pi^+ + p \to \Sigma^+ + p$,B violation,No,No,No
4,$\pi^+ + p \to \Sigma^+ + K^+$,Yes,Yes,Yes,Yes
5,$\pi^+ + p \to \Sigma^+ + p + \pi^0$,B violation,No,No,No
6,$\pi^+ + p \to \Xi^0$,Q violation,No,No,No
7,$\Sigma^+ + p \to \Lambda^0 + \pi^+ + p$,Yes,Yes,Yes,Yes
8,$\Sigma^- + p \to \Xi^- + p$,Yes,"S,I,I3 violation","S,I3 violation",Yes
9,$\Xi^0 \to e^- + p + \bar \nu_e$,Yes,S violation,S violation,Yes
10,$\Xi^0 \to \Lambda^0 + \pi^0$,Yes,"S,I,I3 violation","S,I3 violation",Yes


In [35]:
print(total_result.to_markdown())

|    | reaction                                    | Possibility    | Strong           | EM             | Weak   |
|---:|:--------------------------------------------|:---------------|:-----------------|:---------------|:-------|
|  1 | $p + p \to n + \pi^+ + p$                   | Yes            | Yes              | Yes            | Yes    |
|  2 | $p + p \to \bar n + \pi^+ + p + \pi^0$      | B violation    | No               | No             | No     |
|  3 | $\pi^+ + p \to \Sigma^+ + p$                | B violation    | No               | No             | No     |
|  4 | $\pi^+ + p \to \Sigma^+ + K^+$              | Yes            | Yes              | Yes            | Yes    |
|  5 | $\pi^+ + p \to \Sigma^+ + p + \pi^0$        | B violation    | No               | No             | No     |
|  6 | $\pi^+ + p \to \Xi^0$                       | Q violation    | No               | No             | No     |
|  7 | $\Sigma^+ + p \to \Lambda^0 + \pi^+ + p$    | Yes            | Yes       

### So I get the whole answer

|    | reaction                                    | Possibility    | Strong           | EM             | Weak   |
|---:|:--------------------------------------------|:---------------|:-----------------|:---------------|:-------|
|  1 | $p + p \to n + \pi^+ + p$                   | Yes            | Yes              | Yes            | Yes    |
|  2 | $p + p \to \bar n + \pi^+ + p + \pi^0$      | B violation    | No               | No             | No     |
|  3 | $\pi^+ + p \to \Sigma^+ + p$                | B violation    | No               | No             | No     |
|  4 | $\pi^+ + p \to \Sigma^+ + K^+$              | Yes            | Yes              | Yes            | Yes    |
|  5 | $\pi^+ + p \to \Sigma^+ + p + \pi^0$        | B violation    | No               | No             | No     |
|  6 | $\pi^+ + p \to \Xi^0$                       | Q violation    | No               | No             | No     |
|  7 | $\Sigma^+ + p \to \Lambda^0 + \pi^+ + p$    | Yes            | Yes              | Yes            | Yes    |
|  8 | $\Sigma^- + p \to \Xi^- + p$                | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
|  9 | $\Xi^0 \to e^- + p + \bar \nu_e$            | Yes            | S violation      | S violation    | Yes    |
| 10 | $\Xi^0 \to \Lambda^0 + \pi^0$               | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 11 | $\Xi^- \to \Lambda^0 + \pi^-$               | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 12 | $\Omega^- \to \Lambda^0 + \pi^-$            | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 13 | $\Omega^- \to \Lambda^0 + K^-$              | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 14 | $\Omega^- \to \Xi^0 + \pi^-$                | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 15 | $\Omega^- \to \Xi^0 + K^-$                  | Yes            | Yes              | Yes            | Yes    |
| 16 | $\Xi^- \to \Sigma^0 + \mu^- + \bar \nu_\mu$ | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 17 | $\Xi^- \to e^- + p + \bar \nu_e + \pi^-$    | Yes            | S violation      | S violation    | Yes    |
| 18 | $\Sigma^0 \to \Lambda^0 + \gamma + \gamma$  | Yes            | I violation      | Yes            | Yes    |
| 19 | $\Sigma^0 \to \Lambda^0 + e^- + e^+$        | Yes            | I violation      | Yes            | Yes    |
| 20 | $K^+ \to \mu^+ + \nu_\mu$                   | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 21 | $K^- \to \mu^- + \bar \nu_\mu$              | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 22 | $K^0 \to \pi^+ + \pi^-$                     | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 23 | $K^0 \to \mu^+ + \mu^-$                     | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 24 | $K^+ \to \pi^0 + e^+ + \nu_e$               | Yes            | S,I,I3 violation | S,I3 violation | Yes    |
| 25 | $\mu^+ \to e^+ + \nu_e$                     | L_mu violation | No               | No             | No     |
| 26 | $\pi^+ \to \mu^+ + \bar \nu_\mu$            | L_mu violation | No               | No             | No     |
| 27 | $\pi^- \to \mu^- + \bar \nu_\mu$            | Yes            | I,I3 violation   | I3 violation   | Yes    |